In [43]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

In [44]:
# Load unit path data
unit_path = pd.read_csv('../unit_path.csv')

In [45]:
def generate_graph():
    G = nx.Graph()

    for index, row in unit_path.iterrows():
        source = row['domain']  # 출발 노드
        target = row['range']   # 도착 노드
        edge = row['P']         # 엣지
        weight = row['W']       # 엣지 가중치

        G.add_node(source)      # 출발 노드 추가
        G.add_node(target)      # 도착 노드 추가
        G.add_edge(source, target, property=edge, weight=weight, label=edge)  # 엣지 추가
    
    return G

In [46]:
def heuristic(start_node, target_node):
    w_values = unit_path[(unit_path['domain'] == start_node)]['W'].values
    # The heuristic function of the algorithm is based on the sum of property weights of each path
    if len(w_values) > 0:
        return min(w_values)
    else : 
        return float('inf')

In [57]:
G = generate_graph()

['schema.MusicAlbum', 'foaf.Person', 'schema.Country']

In [58]:
start_node = 'schema.MusicAlbum'
target_node = 'schema.Country'

shortest_path = nx.shortest_path(G, start_node, target_node)
shortest_path

['schema.MusicAlbum', 'foaf.Person', 'schema.Country']